In [71]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import time
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline  
from scipy import stats

from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
import re
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


youtube= pd.read_csv(r'C:\Users\injoo\광주 인공지능 사관학교\로테이션\자연어처리&추천시스템 김준태\미니프로젝트\Dataset\cleaned_USvidoes.csv', error_bad_lines=False)
youtube = youtube.drop_duplicates(['video_id'], keep='first')




In [72]:
youtube.head(1)

,Unnamed: 0,video_id,title,category_id,tags,views,likes,dislikes,comment_total,date,likes_log,views_log,dislikes_log,comment_log
0,0,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,24,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,2017-09-13,7.980024,13.46418,7.694848,6.949856


In [73]:
# youtube['date'] = youtube['date'].apply(lambda x: str(x).replace(".", "-"))
# youtube['date'] = youtube['date'].apply(lambda x: '2017-'+str(x))
# youtube['date'] = pd.to_datetime(youtube['date'], format='%Y-%d-%m')
# youtube['date']
# result = youtube.drop_duplicates('video_id', keep='last').values
# youtube = pd.DataFrame(result, columns = youtube.columns)

In [75]:
only_title = youtube['title']
title_data = only_title

title_data = title_data.str.replace("[^\w]", " ")
#혹시나 공백이 있으면
title_data= title_data.replace('', np.nan)

#결측치 있으면 모두 제거
title_data = title_data.dropna(how='any')

youtube['views_label'] = np.log(youtube['views'] + 1)

# youtube['views_label'] = youtube['views']

# def label_func(x):
#     if x>=17.69:
#         return 1
#     elif x>=12.50:
#         return 2
#     elif x>=11.33:
#         return 3
#     else:
#         return 4

def label_func(x):
    if x>=12.5:
        return 1
    else:
        return 0
    
youtube['views_label'] = youtube['views_label'].apply(lambda x: label_func(x))
train , test, y_train, y_test = train_test_split(title_data, youtube['views_label'], test_size=0.2, shuffle=False)

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
# stop_words = set(stop_words.words('english'))

X_train = []
for stc in train:
    token = []
    words = stc.split()
    for word in words:
        if word not in stop_words:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in test:
    token = []
    words = stc.split()
    for word in words:
        if word not in stop_words:
            token.append(word)
    X_test.append(token)



tokenizer = Tokenizer(7792)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(train)
X_test = tokenizer.texts_to_sequences(test)

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 14
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)



,Unnamed: 0,video_id,title,category_id,tags,views,likes,dislikes,comment_total,date,likes_log,views_log,dislikes_log,comment_log,views_label
0,0,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,24,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,2017-09-13,7.980024,13.464180,7.694848,6.949856,1
1,1,GGm0FQ6i74U,What Hillary Clinton really thinks,25,vox.com|vox|explain|ezra klein|hillary clinton...,244433,9143,6826,4838,2017-09-13,9.120853,12.406701,8.828641,8.484463,0
2,2,qJJHhVf3_ZM,Gwyneth Paltrow Crashes James' Goop Magazine Rant,24,James Corden|The Late Late Show|Colbert|late n...,220661,2070,449,352,2017-09-13,7.635787,12.304387,6.109248,5.866468,0
3,3,9CILSvf1snQ,GOLD DIVA MACARONS- The Scran Line,26,cupcakes|how to make vanilla cupcakes|over the...,27794,1692,34,174,2017-09-13,7.434257,10.232611,3.555348,5.164786,0
4,4,k9bYO7uSnrg,iPhone X hands on,28,iphone x|apple iphone|face id|apple|hardware|h...,14577,160,19,51,2017-09-13,5.081404,9.587269,2.995732,3.951244,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,2359,xlu6i6lT_vk,How Do MASSIVE Sinkholes Form?,27,sinkhole|how do sinkholes form|sinkhole in wat...,440393,14362,390,1575,2017-01-22,9.572411,12.995425,5.968708,7.362645,1
2360,2360,qRoVlH1OcI4,Trump slams Clinton for defending NFL anthem p...,25,Business Insider|Donald Trump|Hillary Clinton|...,55762,1265,760,1873,2017-01-22,7.143618,10.928866,6.634633,7.535830,0
2361,2361,EoejGgUNmVU,LP - Lost On You (A Night at The McKittrick Ho...,10,LP|Death Valley|Other People|Lost On You|The M...,142908,7088,68,437,2017-01-22,8.866300,11.869963,4.234107,6.082219,0
2362,2362,MT1CMTI0EVw,Tré Melvin @ #YouTubeBlack FanFest Washington ...,24,YouTube FanFest|#YTFF|Washington DC|USA|YTFF|#...,24532,2148,77,0,2017-01-22,7.672758,10.107774,4.356709,0.000000,0


In [76]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, LSTM
model = Sequential()
model.add(Embedding(5000, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='relu'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics= 'accuracy' )
# model.compile(loss='categorical_crossentropy' ,optimizer='adam' ,metrics= 'accuracy' )
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
 1/60 [..............................] - ETA: 0s - loss: 4.3010 - accuracy: 0.5312

InvalidArgumentError:  indices[27,7] = 5015 is not in [0, 5000)
	 [[node sequential_3/embedding_3/embedding_lookup (defined at <ipython-input-76-e5144ac8506a>:10) ]] [Op:__inference_train_function_15212]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_3/embedding_3/embedding_lookup:
 sequential_3/embedding_3/embedding_lookup/14000 (defined at C:\Users\injoo\Anaconda3\lib\contextlib.py:112)

Function call stack:
train_function


In [4]:
sentence = input()
# 토큰화
token_stc = sentence.split()
# 정수 인코딩
encode_stc = tokenizer.texts_to_sequences([token_stc])
# 패딩
pad_stc = pad_sequences(encode_stc, maxlen = 14)

score = model.predict(pad_stc)
print(score)

iphone review
[[0.7711972]]


In [5]:
sentence = input()
# 토큰화
token_stc = sentence.split()
# 정수 인코딩
encode_stc = tokenizer.texts_to_sequences([token_stc])
# 패딩
pad_stc = pad_sequences(encode_stc, maxlen = 14)

score = model.predict(pad_stc)
print(score)

my vlog
[[0.27767742]]


In [6]:
model.evaluate(X_test, y_test)

15/15 [==============================] - 0s 997us/step - loss: 2.4780 - accuracy: 0.6321


[2.478018283843994, 0.6321353316307068]

## 여기서부터가 오늘 새로 하는 것

In [7]:
df= pd.read_csv(r'C:\Users\injoo\광주 인공지능 사관학교\로테이션\자연어처리&추천시스템 김준태\미니프로젝트\Dataset\one_hot_encoded.csv', error_bad_lines=False)


In [8]:
type(title_data)

pandas.core.series.Series

In [9]:
def pred_func(x):
    if model.predict(x) >= 0.5:
        return 1
    else:
        return 0

In [10]:
title_data.shape

(2364,)

In [11]:
youtube.shape

(2364, 12)

In [78]:
youtube['views_label']

0       1
1       0
2       0
3       0
4       0
       ..
2359    1
2360    0
2361    0
2362    0
2363    0
Name: views_label, Length: 2364, dtype: int64

In [79]:
one_hot =pd.read_csv(r'C:\Users\injoo\광주 인공지능 사관학교\로테이션\자연어처리&추천시스템 김준태\미니프로젝트\Dataset\one_hot_encoded.csv', error_bad_lines=False)


In [87]:
one_hot

,Unnamed: 0,likes_log,views_log,dislikes_log,comment_log,category_id_Autos & Vehicles,category_id_Comedy,category_id_Education,category_id_Entertainment,category_id_Film & Animation,...,category_id_Howto & Style,category_id_Music,category_id_News & Politics,category_id_Nonprofits & Activism,category_id_People & Blogs,category_id_Pets & Animals,category_id_Science & Technology,category_id_Shows,category_id_Sports,category_id_Travel & Events
0,0,7.980024,13.464180,7.694848,6.949856,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,9.120853,12.406701,8.828641,8.484463,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2,7.635787,12.304387,6.109248,5.866468,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,7.434257,10.232611,3.555348,5.164786,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,5.081404,9.587269,2.995732,3.951244,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,2359,9.572411,12.995425,5.968708,7.362645,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2360,2360,7.143618,10.928866,6.634633,7.535830,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2361,2361,8.866300,11.869963,4.234107,6.082219,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2362,2362,7.672758,10.107774,4.356709,0.000000,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
youtube['views_label']

0       1
1       0
2       0
3       0
4       0
       ..
2359    1
2360    0
2361    0
2362    0
2363    0
Name: views_label, Length: 2364, dtype: int64

In [89]:
view_dummies = pd.get_dummies(youtube['views_label'])

In [90]:
view_dummies.columns = ['unpopular', 'popular']

In [91]:
view_dummies

,unpopular,popular
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
2359,0,1
2360,1,0
2361,1,0
2362,1,0


In [94]:
new_df = pd.concat([one_hot, view_dummies], axis=1)

In [95]:
new_df

,Unnamed: 0,likes_log,views_log,dislikes_log,comment_log,category_id_Autos & Vehicles,category_id_Comedy,category_id_Education,category_id_Entertainment,category_id_Film & Animation,...,category_id_News & Politics,category_id_Nonprofits & Activism,category_id_People & Blogs,category_id_Pets & Animals,category_id_Science & Technology,category_id_Shows,category_id_Sports,category_id_Travel & Events,unpopular,popular
0,0,7.980024,13.464180,7.694848,6.949856,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,9.120853,12.406701,8.828641,8.484463,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,2,7.635787,12.304387,6.109248,5.866468,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,3,7.434257,10.232611,3.555348,5.164786,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,5.081404,9.587269,2.995732,3.951244,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,2359,9.572411,12.995425,5.968708,7.362645,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2360,2360,7.143618,10.928866,6.634633,7.535830,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2361,2361,8.866300,11.869963,4.234107,6.082219,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2362,2362,7.672758,10.107774,4.356709,0.000000,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [96]:
new_df.drop(columns=['Unnamed: 0'])

,likes_log,views_log,dislikes_log,comment_log,category_id_Autos & Vehicles,category_id_Comedy,category_id_Education,category_id_Entertainment,category_id_Film & Animation,category_id_Gaming,...,category_id_News & Politics,category_id_Nonprofits & Activism,category_id_People & Blogs,category_id_Pets & Animals,category_id_Science & Technology,category_id_Shows,category_id_Sports,category_id_Travel & Events,unpopular,popular
0,7.980024,13.464180,7.694848,6.949856,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,9.120853,12.406701,8.828641,8.484463,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,7.635787,12.304387,6.109248,5.866468,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,7.434257,10.232611,3.555348,5.164786,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5.081404,9.587269,2.995732,3.951244,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,9.572411,12.995425,5.968708,7.362645,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2360,7.143618,10.928866,6.634633,7.535830,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2361,8.866300,11.869963,4.234107,6.082219,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2362,7.672758,10.107774,4.356709,0.000000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [114]:
new_df.to_csv(r'./\final_cleaned.csv')

# 여기서부터가 진짜!

In [113]:
new_df_columns = new_df.columns.values
column_list = list(new_df_columns)
print(column_list)
column_list = column_list.remove('views_log')

['Unnamed: 0', 'likes_log', 'views_log', 'dislikes_log', 'comment_log', 'category_id_Autos & Vehicles', 'category_id_Comedy', 'category_id_Education', 'category_id_Entertainment', 'category_id_Film & Animation', 'category_id_Gaming', 'category_id_Howto & Style', 'category_id_Music', 'category_id_News & Politics', 'category_id_Nonprofits & Activism', 'category_id_People & Blogs', 'category_id_Pets & Animals', 'category_id_Science & Technology', 'category_id_Shows', 'category_id_Sports', 'category_id_Travel & Events', 'unpopular', 'popular']


In [111]:
print(column_list)

None


In [138]:
views=new_df['views_log']
youtube_view2 = new_df[['likes_log','dislikes_log', 'comment_log', 'category_id_Autos & Vehicles', 'category_id_Comedy', 'category_id_Education', 'category_id_Entertainment', 'category_id_Film & Animation', 'category_id_Gaming', 'category_id_Howto & Style', 'category_id_Music', 'category_id_News & Politics', 'category_id_Nonprofits & Activism', 'category_id_People & Blogs', 'category_id_Pets & Animals', 'category_id_Science & Technology', 'category_id_Shows', 'category_id_Sports', 'category_id_Travel & Events', 'popular', 'unpopular']]
train,test,y_train,y_test=train_test_split(youtube_view2,views, test_size=0.2,shuffle=False)

In [140]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
nEstimator = [140,160,180,200,220]
depth = [10,15,20,25,30]

RF = RandomForestRegressor()
hyperParam = [{'n_estimators':nEstimator,'max_depth': depth}]
grid = GridSearchCV(RF,hyperParam,cv=5,verbose=1,scoring='r2',n_jobs=-1)
grid.fit(train,y_train)


Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   16.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [142]:
print("Best HyperParameter: ",grid.best_params_)

Best HyperParameter:  {'max_depth': 10, 'n_estimators': 200}


In [147]:
model = RandomForestRegressor(n_estimators = 200,max_depth= 10)
model.fit(train, y_train)
print("훈련 세트 점수: {:.2f}".format(model.score(train, y_train)))
print("테스트 세트 점수: {:.2f}".format(model.score(test, y_test)))

훈련 세트 점수: 0.96
테스트 세트 점수: 0.86


In [149]:

# linearRegression
model = LinearRegression()
model.fit(train, y_train)
print("훈련 세트 점수: {:.2f}".format(model.score(train, y_train)))
print("테스트 세트 점수: {:.2f}".format(model.score(test, y_test)))
print("사용한 특성의 수: {}".format(   np.sum( model.coef_ != 0 )   ))

훈련 세트 점수: 0.84
테스트 세트 점수: 0.84
사용한 특성의 수: 21


In [150]:
from sklearn.linear_model import Ridge
model = Ridge()
model.fit(train, y_train)
print("훈련 세트 점수: {:.2f}".format(model.score(train, y_train)))
print("테스트 세트 점수: {:.2f}".format(model.score(test, y_test)))
print("사용한 특성의 수: {}".format(   np.sum( model.coef_ != 0 )   ))

훈련 세트 점수: 0.84
테스트 세트 점수: 0.84
사용한 특성의 수: 21
